## Environment variables 

In [0]:
import os

In [0]:
# current directory
os.getcwd()

In [0]:
os.path.exists('/Workspace/Users/arundhatidaschowdhury@gmail.com/data/SampleDataFile.json')

In [0]:
# listing using os.listdir
# filtering using list comprehension 
linked_in_files = [f"data/{file}" for file in os.listdir("data") if file.startswith('linked')]
linked_in_files

In [0]:
# get the timestamp which I need for sorting
# Sort using the timestamp - descending
# Get index 0

date = [(i[i.index("_", i.index("_") + 1) + 1:-5],i) for i in linked_in_files]
#print(sorted(date, reverse=True))
print(max(date)[1])

In [0]:
# all together in a function
def get_latest_file(base_path):
    linked_in_files = [f"data/{file}" for file in os.listdir(base_path) if file.startswith('linked')]
    date = [(i[i.index("_", i.index("_") + 1) + 1:-5],i) for i in linked_in_files]
    return max(date)[1]

In [0]:
get_latest_file("data")

In [0]:
x = 'data/linkedin_file1_2025_01_01.json'
# x.split("/")
x.split(".")[0][-10:]

In [0]:
linked_in_files

In [0]:
val = lambda x: x+1
val(1)

In [0]:
max(linked_in_files, key = lambda x: x.split(".")[0][-10:])

In [0]:
values = [ (4,6), (2,1), (8,3)] 
# By default, the sort will be done on first element of the tuple
sorted( values )

# But how can we sort using the 2nd element? 
# 2nd eleemnt of each tuple
sorted( values , key = lambda x : x[1] )

In [0]:
a = ["abcdefg", "hi", "jkl"]
# "hi", "jkl", "abcedfg"
sorted(a, key = lambda x: len(x))

## Test API

- Read json file /Workspace/Users/arundhatidaschowdhury@gmail.com/data/SampleDataFile.json
- Scan through each record


In [0]:
# import pandas as pd
# df = pd.read_json(file)
# display(df)

In [0]:
file = "/Workspace/Users/arundhatidaschowdhury@gmail.com/data/SampleDataFile.json"

In [0]:
# Read the json file
import json

with open(file) as f:
    data=f.read()
print(data)

json_data=json.loads(data)
print(type(json_data))

In [0]:
for record in json_data:
    original_record = record
    original_address = record.get('address')
    print(original_address)

In [0]:
json_addr=[i["address"] for i in json_data]

#addr_concate=[i["add1"]+" "+i["add2"]+" "+i["city"]+" "+i["state"] for i in json_addr]

addr_concate=[" ".join(i.values()) for i in json_addr]
print(addr_concate)

In [0]:
import requests
from pprint import pprint

API_KEY = "77625a8592944180861ba4c6d9008328"
url = "https://api.geoapify.com/v1/geocode/autocomplete"

for i in addr_concate:
    params = {
    "text": i,
    "apiKey": API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    pprint(data["features"][0]["properties"])
    #["postcode"])

- Read file 
- Convert to Json - json.dumps
- API request call
- API response parse 

```
def read_json_file(file_path):
  .......
  return <json> # will be a list of dict


record_json = read_json_file(path)

Each record in this record_json is : 

  {
    "customer_id": "CUST004",
    "customer_name": "Shashi Tharoor",
    "source": "Call",
    "address": {
      "add1": "13048 Zion Dr",
      "add2": "",
      "city": "Providence Village",
      "state": "TX",
      "zip": ""
    }
  }

Objective is to get each record in this format: 

  {
    "customer_id": "CUST004",
    "customer_name": "Shashi Tharoor",
    "source": "Call",
    "address": {
      "add1": "13048 Zion Dr",
      "add2": "",
      "city": "Providence Village",
      "state": "TX",
      "zip": ""
    },
    "address_enriched": {
      "add1": "13048 Zion Dr",
      "add2": "",
      "city": "Providence Village",
      "state": "TX",
      "zip": ""
    }
  }


  x["address_enriched"] = < get from API response > 

  for rec in record_json:
    - extract address into some addr
    - def enrich_address(addr_dct:dict) -> dict : 
      - convert addr to string
      - call API
      - get resoponse 
      - update zipcode
      - return updated addr
=

```

In [0]:
json.load (load from a file pointer)
json.loads (load from a string representation of json)

json.dump (writing to a file pointer)
json.dumps (writing to a string)

In [0]:
# def read_json_file(file_path):
#     with open(file_path) as f:
#         data=f.read()
#         json_data=json.loads(data)
#     return(json_data)

def read_json_file(file_path):
    with open(file_path) as f:
        data=json.load(f)
    return(data)
read_json_file(file)

In [0]:
import json
import requests

file = "/Workspace/Users/arundhatidaschowdhury@gmail.com/data/SampleDataFile.json"
API_KEY = "77625a8592944180861ba4c6d9008328"
url = "https://api.geoapify.com/v1/geocode/autocomplete"

def read_json_file(file_path):
    with open(file_path) as f:
        data=f.read()
        json_data=json.loads(data)
    return(json_data)

def call_api(addr_text):
    params = {
        "text": addr_text,
        "apiKey": API_KEY
        }
    response = requests.get(url, params=params)
    data = response.json()
    return data


def process_data(file_path):
    # Step 1: Read json file and get all full customer records 
    json_data = read_json_file(file_path)
    new_addr_list = []
    # i is a full customer record
    for i in json_data:
        # Get the address part of the record
        addr_i = i["address"]
        # From the address part make the text concat 
        addr_text = " ".join(addr_i.values())
        # Call API to get new address
        api_response = call_api(addr_text)
        # Extract new address from API response 
        new_data=api_response["features"][0]["properties"]
        # pprint(new_data)
        i["address_enriched"] = {
                "add1": new_data["address_line1"],
                "add2": "",
                "city": new_data["city"],
                "state": new_data["state_code"],
                "zip": new_data["postcode"] 
                }
        new_addr_list.append(i)
        
    return(new_addr_list)
    
process_data(file)

In [0]:
new_file = "/Workspace/Users/arundhatidaschowdhury@gmail.com/data/SampleDataFileWrite.json"

def write_json_file(file_path_read,file_path_write):
    with open(file_path_write,"w") as f:
        json_write_data=json.dump(process_data(file_path_read),f)
    return(json_write_data)

write_json_file(file,new_file)